<a href="https://www.kaggle.com/code/amirmotefaker/sentiment-analysis-on-tiktok?scriptVersionId=128311172" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction
- TikTok is one of the most popular social media apps today. It is famous for its short videos. People often use this app to watch entertaining and funny videos. Despite its entertaining content, this application is not liked by everyone. If you skim through its reviews, you’ll find a mixture of hate and support for TikTok all over the world. Let’s analyze TikTok reviews to find out what people feel about the content of this app.